In [ ]:
!pip install kora q
import kora.install.rdkit

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/code/generator')
sys.path.append('/content/gdrive/MyDrive/code/predictor')
sys.path.append('/content/gdrive/MyDrive/code/reinforce')
sys.path.append('/content/gdrive/MyDrive/code/result')
sys.path.append('/content/gdrive/MyDrive/code/dataset')



In [ ]:
import argparse
import os
import numpy as np
import pandas as pd
import random
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import RDLogger 


from models import RNN, OneHotRNN, EarlyStopping
from datasets import SmilesDataset, SelfiesDataset, SmilesCollate
from functions import decrease_learning_rate, print_update, track_loss, \
     sample_smiles, write_smiles

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#Load the Pre-trained Generator

In [ ]:
def load_model(model, path):
        weights = torch.load(path)
        model.load_state_dict(weights)


def is_valid(smiles):
  mol = Chem.MolFromSmiles(smiles)
  if mol is not None and mol.GetNumAtoms()>0:
    return smiles

import warnings
warnings.filterwarnings("ignore")
RDLogger.DisableLog('rdApp.*') # switch off RDKit warning messages

In [ ]:
dataset = SmilesDataset(smiles_file='/content/gdrive/MyDrive/code/generator/pre-trained/chembl_500000.csv', vocab_file ='/content/gdrive/MyDrive/code/generator/pre-trained/vocab_chembl_500000_pat_50000')


print(dataset.vocabulary)
seed = 0
batch_size = 128
## seed all RNGs
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    print("using cuda")
    torch.cuda.manual_seed_all(seed)

loader = DataLoader(dataset,
                    batch_size=batch_size,
                    shuffle=True,
                    drop_last=True,
                    collate_fn=SmilesCollate(dataset.vocabulary))

model = RNN(vocabulary=dataset.vocabulary,
                rnn_type='GRU',                      # str; RNN type choices=['RNN', 'LSTM', 'GRU']
                embedding_size= 128,                 # int; embedding size
                hidden_size=512,                     # int; size of language model hidden layers
                n_layers=3,                          # int; number of layers in language model
                dropout=0,                           # float; amount of dropout (0-1) to apply to model
                bidirectional=False,                 # bool; for LSTMs only, train a bidirectional mode
                tie_weights=False,
                nonlinearity='tanh')
# Print model's state_dict
#print("Model's state_dict:")
#for param_tensor in model.state_dict():
    #print(param_tensor, "\t", model.state_dict()[param_tensor].size())


model_path = '/content/gdrive/MyDrive/code/generator/pre-trained/checkpoint_chembl_500000_pat_50000'
load_model(model, model_path)

sampled_smiles = []

sample_size = 500
while len(sampled_smiles) < sample_size:
    sampled_smiles.extend(model.sample(batch_size, return_smiles=True))


mols = list(filter(is_valid,sampled_smiles)) # Valid
print("Percentage of validity for pre-trained generator: " + str((len(mols)/len(sampled_smiles))*100))


#Load Predictor Model

In [ ]:
!git clone https://github.com/fastai/fastai1.git

In [ ]:
!mv /content/fastai1 /content/fastai_pred1

In [ ]:
!mv /content/fastai_pred1/fastai /content/fastai_pred1/fastai_pred

In [ ]:
import sys
sys.path.append('/content/fastai_pred1')
sys.path.append('/content/fastai_pred1/fastai_pred')

In [ ]:


from sklearn.model_selection import train_test_split


from fastai_pred import *
from fastai_pred.text import *
from fastai_pred.vision import *
from fastai_pred.imports import *


import numpy as np
import threading
import random
from sklearn.utils import shuffle
import pandas as pd 
import numpy as np


import os
current_path = os.getcwd()
print(current_path)

In [ ]:
import tl_Predictor_Reaction_c
from tl_Predictor_Reaction_c import pred_init, train_reg, test_performance, test_performance, predictor 

In [ ]:
#Parameter defining
seed_tl = 1234
batch_size = 128
filename = pd.read_csv('/content/gdrive/MyDrive/code/dataset/Reaction_c.csv')
augm = 100
drp_out = 0.2 
sigm_g = 0.5


In [ ]:
#Loading of pre-trained weight using Transfer Learning
reg_learner_pre, train_aug , valid = pred_init(seed_tl, batch_size, filename, current_path, augm, drp_out, sigm_g)

In [ ]:
test_rmse = test_performance(seed_tl, batch_size, filename, train_aug, valid, current_path, drp_out, sigm_g)

#Fragment Reinforcement Learning

In [ ]:
import numpy as np
from tqdm import tqdm, trange
import pickle
from rdkit import Chem, DataStructs
from rdkit.Chem import MACCSkeys
from rdkit.Chem import AllChem, DataStructs
import random

import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ExponentialLR, StepLR
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
import functions_rl
from functions_rl import generate_allfragments, join_frag, usable_frag, permute, gen_firstatom_frag, plot_hist, tensor_to_array, canonical_smiles

In [ ]:
def add_othercomponents(smile, components = 'C=C(CC(=O)OC)C(=O)OC.ClCCl'):
    return smile + '.' + components

In [ ]:
def novelty_score(mols,ref_mols): 
    return set.difference(mols,ref_mols)

In [ ]:
def dataframe(smile, pred_val):
  smile_df = pd.DataFrame(smile, columns = ['smiles'])
  prediction_array = list(tensor_to_array(pred_val))
  prediction_df = pd.DataFrame(prediction_array, columns = ['predicted_value'])
  smile_pred_df  = pd.concat([smile_df,prediction_df], axis =1)
  return smile_pred_df
 

In [ ]:
smiles_train_file= pd.read_csv('/content/gdrive/MyDrive/code/generator/pre-trained/chembl_500000.csv', header= None)
smiles_train_file.columns = ['smiles_train']
#reference dataset
obj_ref = list(set(smiles_train_file.smiles_train))
print(len(obj_ref))

In [ ]:
def estimate_and_update(gen, tl_Predictor_Reaction_c, n_to_generate, core_smi='P(*)1OC2=C(C3=C(C=CC=C4)C4=CC=C3O1)C5=CC=CC=C5C=C2', **kwargs):

    rng = np.random.default_rng()
    seed_value = rng.integers(low = 10000)

    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value) # gpu vars
    torch.backends.cudnn.deterministic = True  #needed
    torch.backends.cudnn.benchmark = False

    generated = []
    generated_mol = []
    pbar = tqdm(range(n_to_generate))
    for i in pbar:
        pbar.set_description("Generating molecules...")
        no_sample = 1
        sampled_smiles = gen.sample(no_sample, max_len=100, return_smiles=True)
        generated.append(sampled_smiles)
        
    generated = [ y for ys in generated for y in ys]
  
    generated_novel = []
    
    x = 0

    for j in range(len(generated)):
        if_smile = Chem.MolFromSmiles(generated[j])
        if if_smile is not None:
            x+=1

            fragment = gen_firstatom_frag(generated[j])
            mol = core_smi
            for i in range(core_smi.count('(*)')):
                mol = join_frag(mol, fragment)
                mol = usable_frag(mol)

            generated_novel.append(generated[j])
            generated_mol.append(mol)
            
    if x==0:
        return [], []

    sanitized = canonical_smiles(generated_mol, sanitize=False, throw_warning=False)[:-1]
    unique_smiles = list(np.unique(sanitized))[1:]

    unique_components = []
    for i in range(len(unique_smiles)):
        unique_components.append(add_othercomponents(unique_smiles[i]))


    smiles, prediction = tl_Predictor_Reaction_c.predictor(unique_components, seed_tl, batch_size, filename, train_aug, valid, current_path, drp_out, sigm_g)
    
    novel_mols = novelty_score(set(generated_novel), set(obj_ref))

    print("Total number of valid fragment backbones generated:", x)
    print("Percentage of validity:", (x/n_to_generate)*100)
    print("Percentage of uniqueness", (len(set(generated_novel))/n_to_generate)*100)
    print("Percentage of novelty", (len(novel_mols)/n_to_generate)*100)                                            
    plot_hist(prediction, n_to_generate)

    return smiles, prediction

**Unbiased Generation**

In [ ]:
smiles_unbiased, prediction_unbiased =estimate_and_update(model, tl_Predictor_Reaction_c,
                                                              n_to_generate=500)

In [ ]:
smile_pred_unbiased_df = dataframe(smiles_unbiased, prediction_unbiased)
smile_pred_unbiased_df.to_csv('/content/gdrive/MyDrive/code/result/unbiased_Recation_c.csv')

**Biased Generation**

In [ ]:
import fragment_rl_Reaction_c
from fragment_rl_Reaction_c import Reinforcement

In [ ]:
n_to_generate = 500
n_policy_replay = 10
n_policy = 5
n_iterations = 10

In [ ]:
def simple_moving_average(previous_values, new_value, ma_window_size=10):
    value_ma = np.sum(previous_values[-(ma_window_size-1):]) + new_value
    value_ma = value_ma/(len(previous_values[-(ma_window_size-1):]) + 1)
    return value_ma

In [ ]:
def get_reward(smiles, tl_Predictor_Reaction_c, invalid_reward=0.0):
    rewards = np.zeros([len(smiles)])
    
    mol, prop = tl_Predictor_Reaction_c.predictor(smiles, seed_tl, batch_size, filename, train_aug, valid, current_path, drp_out, sigm_g)

    for i in range(len(smiles)):
        if smiles[i] is '':
            rewards[i] = -2
        else:
            pred = tensor_to_array(prop)
            if pred[i] != pred[i]:
                rewards.append(invalid_reward)
            else:
                t=int((pred[i]-50)/5)
                if t<0:
                    t=0
                rewards[i] = ((t*2)+1)
    return rewards

In [ ]:
def get_pred_val(smiles, tl_Predictor_Reaction_c):
    generated_novel = []
    for j in range(len(smiles)):
        if_smile = Chem.MolFromSmiles(smiles[j])
        if if_smile is not None:
            generated_novel.append(smiles[j])  
    unique_components = list(np.unique(generated_novel))
    mol, prop = tl_Predictor_Reaction_c.predictor(unique_components, seed_tl, batch_size, filename, train_aug, valid, current_path, drp_out, sigm_g)
    prop_new = tensor_to_array(prop)
    return prop_new

In [ ]:
RL_max = Reinforcement(model, tl_Predictor_Reaction_c, get_reward, get_pred_val)

In [ ]:
rewards_max = []
rl_losses_max = []
pred_Reinforce_max_plot = []

In [ ]:
for i in range(n_iterations):
    for j in trange(n_policy, desc='Policy gradient...'):
        cur_reward, cur_loss, cur_pred = RL_max.policy_gradient(dataset.vocabulary)
        pred_Reinforce_max_plot.append(cur_pred)
        rewards_max.append(simple_moving_average(rewards_max, cur_reward)) 
        rl_losses_max.append(simple_moving_average(rl_losses_max, cur_loss))
        
    plt.plot(rewards_max)
    plt.xlabel('Training iteration')
    plt.ylabel('Average reward')
    plt.show()
    plt.plot(rl_losses_max)
    plt.xlabel('Training iteration')
    plt.ylabel('Loss')
    plt.show()
    plt.plot(pred_Reinforce_max_plot)
    plt.xlabel('Training iteration')
    plt.ylabel('Predicted value')
    plt.show()

    

    smiles_cur, prediction_cur =estimate_and_update(RL_max.generator, tl_Predictor_Reaction_c,
                                                    n_to_generate=n_to_generate)
    print('Sample trajectories:')
    for sm in smiles_cur[:5]:
        print(sm)


In [ ]:
smiles_bias500, prediction_bias500 =estimate_and_update(RL_max.generator, tl_Predictor_Reaction_c, n_to_generate=500)

In [ ]:
smile_pred_biased_df = dataframe(smiles_bias500, prediction_bias500)
smile_pred_biased_df.to_csv('/content/gdrive/MyDrive/code/result/biased_Recation_c.csv')